#### Componentes Fundamentais:

- **Models (Modelos):** LangChain serve como uma interface padrão que permite interações com uma ampla gama de Grandes Modelos de Linguagem (LLMs).

- **Chains (Cadeias):** Como seu nome implica, _cadeias_ são o núcleo dos fluxos de trabalho do LangChain. Combinam LLMs com outros componentes, criando aplicativos por meio da execução de uma sequência de funções.

- **Prompts (Instruções):** Os prompts são as instruções apresentadas a um LLM. Geralmente, a "arte" de redigir prompts que efetivamente entregam o contexto necessário para que o LLM interprete a entrada e a saída da estrutura da maneira mais útil para você é chamada de engenharia de prompt.

- **Indexes (Índices):** Para realizar determinadas tarefas, as LLMs precisarão acessar fontes de dados externas específicas não incluídas em seu conjunto de dados de treinamento, como documentos internos, e-mails ou conjuntos de dados. LangChain refere-se coletivamente a essa documentação externa como “índices"."

- **Memory (Memória):** Por padrão, os LLMs não têm memória de longo prazo de conversas anteriores (a menos que o histórico do chat seja usado como entrada para uma consulta). O LangChain soluciona esse problema com utilitários simples para adicionar memória a um sistema, com opções que vão desde a retenção total de todas as conversas até a retenção de um resumo da conversa até a retenção das _n_ trocas mais recentes.

- **Agents/Tools (Agentes/Ferramentas):** Os agentes do LangChain podem usar um determinado modelo LLM como um "mecanismo de raciocínio" para determinar quais ações tomar. Ao criar uma cadeia para um agente, as entradas contêm:

	- uma lista de ferramentas disponíveis para serem aproveitadas.
	- entrada do usuário (como prompts e consultas).
	- quaisquer etapas relevantes executadas anteriormente.




In [ ]:
%run ../helpers/00-llm.ipynb

In [ ]:
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate
from langchain_core.messages import HumanMessage

# Importa conectando com o cliente OpenAI
from helpers.llm import initialize_llm, logger, pretty_print

# Parte 1 - Importando os componentes do LangChain
llm, _, _ = initialize_llm()

In [ ]:
print("=========== Exemplo 1 =========")

prompt_template = PromptTemplate.from_template("Gere para mim um poema sobre: {assunto}. Escreva em {lingua}")
retorno = prompt_template.invoke({"assunto": "navegação", "lingua":"italiano"})

pretty_print(retorno)
print("-------------------------------------------------------------------")
 

In [ ]:
print("=========== Exemplo 2 =========")

template = '''
Você é um analista financeiro.
Escreva um relatório financeiro detalhado para a empresa "{empresa}" para o período {periodo}.

O relatório deve ser escrito em {idioma} e incluir as seguintes análises:
{analises}

Certifique-se de fornecer insights e conclusões para cada seção.
'''

prompt_template = PromptTemplate.from_template(template=template)

empresa = 'ACME Corp'
periodo = 'Q1 2024'
idioma = 'Português'
analises = [
    "Análise do Balanço Patrimonial",
    "Análise do Fluxo de Caixa",
    "Análise de Tendências",
    "Análise de Receita e Lucro",
    "Análise de Posição de Mercado"
]
analises_text = "\n".join([f"- {analise}" for analise in analises])

print(analises_text)

In [ ]:
prompt = prompt_template.format(
    empresa=empresa,
    periodo=periodo,
    idioma=idioma,
    analises=analises_text
)
print("Prompt Gerado:\n", prompt)

response = llm.invoke(prompt)
pretty_print(response)

In [ ]:
print("=========== Exemplo 3 =========")

prompt_template = ChatPromptTemplate([
									  ("system", "Você é um assistente de IA com habilidade de escritor de poesia."),
									  ("user", "Gere para mim um poema sobre: {assunto}. Escreva em {lingua}")
])

retorno = prompt_template.invoke({"assunto": "navegação", "lingua":"italiano"})
pretty_print(retorno)
print("-------------------------------------------------------------------")

resposta = llm.invoke(retorno)

pretty_print(resposta.content)
print("-------------------------------------------------------------------")

In [12]:
### Exemplo 1:
# PART 1: Criando ChatPromptTemplate
print("-----Exemplo Chain 1 -----")

prompt_template = ChatPromptTemplate([
    ("user", "Escreva um poema em {lingua} sobre o tema: {assunto}")
]
)

# PART 2: Criando a chain
chain1 = prompt_template | llm

# PART 3: Invoke da chain passando as variáveis.
resposta = chain1.invoke({"lingua": "pt-br", "assunto":"frutas"})

pretty_print(resposta.content)
print("-------------------------------------------------------------------")

-----Exemplo Chain 1 -----


INFO:httpx:HTTP Request: POST https://trainning-grassato-01.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"



----------------------------------------------------------------------------------------------------

No pomar da manhã, o sol desperta,  
Entre folhas e galhos, a vida é certa.  
Frutas pendem, coloridas, no ar,  
Promessa doce de se saborear.

A manga dourada, perfume no vento,  
O cacho de uva, segredo e alento.  
A maçã reluzente, tentação no olhar,  
Banana macia, pronta pra descascar.

O abacaxi, coroa de rei,  
Ácido e doce, mistura que amei.  
A melancia, frescor no verão,  
Gotas vermelhas, festa na mão.

Laranja e limão, cítricos irmãos,  
Explodem sabores, despertam paixões.  
Mamão e goiaba, doçura sem fim,  
No pomar da vida, tudo é jardim.

Frutas são cores, são risos, são festa,  
São bênçãos da terra, banquete que resta.  
No prato, no suco, no cheiro, no chão,  
Frutas são versos do nosso verão.
-------------------------------------------------------------------


In [ ]:
### Exemplo 2:
# PART 1: Criando ChatPromptTemplate já com mensagem de sistema:
print("-----Exemplo Chain 2 -----")

mensagens = [
    ("system", "Você é um poeta brasileiro famoso e escreve poemas de no máximo {n_versos} versos."),
    ("human", "Escreva para mim um poema sobre {assunto}."),
]

prompt_template = ChatPromptTemplate(mensagens)

# PART 2: Criando a chain
chain2 = prompt_template | llm

# PART 3: Invoke da chain passando as variáveis.
resposta = chain2.invoke({"n_versos": "10", "assunto":"navios"})

pretty_print(resposta.content)


print("-------------------------------------------------------------------")


resposta = llm.invoke("Qual a capital da França?")
pretty_print(resposta)